# How to build simple LLM App with LangChain

## Goal of App
* Translate text from English into another language

## Concepts included
* Monitor with LangSmith
* Connect with LLM
* Use Prompt Template
* Use Output Parser
* Chain Prompt Template, LLM call and Output Parser
* Deploy with LangServe

In [1]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]

In [2]:
MODEL_GPT = 'gpt-4o-mini'

## Connect with LLM and ask it to translate sentence from English to Spanish

In [3]:
from langchain_openai import ChatOpenAI

# llm = ChatOpenAI(model="gpt-3.5-turbo")
llm = ChatOpenAI(model=MODEL_GPT)

In [4]:
from langchain_core.messages import HumanMessage, SystemMessage

messagesToTheLLM = [
    SystemMessage(content="Translate the following from English into Spanish"),
    HumanMessage(content="Generative AI is the greatest value-creation opportunity in Human History."),
]

### Step 1: call LLM

In [5]:
llm.invoke(messagesToTheLLM)

AIMessage(content='La inteligencia artificial generativa es la mayor oportunidad de creación de valor en la historia de la humanidad.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 32, 'total_tokens': 53, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_06737a9306', 'finish_reason': 'stop', 'logprobs': None}, id='run-3b064189-182d-49e6-9cd3-b3aff58188b2-0', usage_metadata={'input_tokens': 32, 'output_tokens': 21, 'total_tokens': 53, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

### Track operation in LangSmith
* [Open LangSmith here](https://smith.langchain.com/)

## Use Output Parser to format response of LLM as string of text

In [6]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

### Step 1: call LLM

In [7]:
initialResponse = llm.invoke(messagesToTheLLM)

### Step 2: apply parser to format output

In [8]:
formattedResponse = parser.invoke(initialResponse)

In [9]:
print(formattedResponse)

La inteligencia artificial generativa es la mayor oportunidad de creación de valor en la historia de la humanidad.


## Chain the LLM Call and the Output Parser

In [10]:
llmAndParserChained = llm | parser

### Steps 1 and 2: call LLM and apply parser to output

In [11]:
responseFromChain=llmAndParserChained.invoke(messagesToTheLLM)

In [12]:
print(responseFromChain)

La inteligencia artificial generativa es la mayor oportunidad de creación de valor en la historia de la humanidad.


## Use Prompt Template with System Message and Variables

In [13]:
from langchain_core.prompts import ChatPromptTemplate

In [14]:
system_template = "Translate the following text input into {language}:"

In [15]:
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", system_template), 
        ("user", "{text_input}")
    ]
)

In [16]:
SecondChain = prompt_template | llm | parser

In [17]:
ResponseFromTheLLM = SecondChain.invoke(
    {
    "language": "Spanish",
    "text_input": "Now is the moment to learn Generative AI!"
    }
)

In [18]:
print(ResponseFromTheLLM)

¡Ahora es el momento de aprender sobre IA Generativa!


## Deploy App using LangServe

In [19]:
# !pip install "langserve[all]"

In [21]:
from fastapi import FastAPI
from langserve import add_routes
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv, find_dotenv

import os

_ = load_dotenv(find_dotenv())

openai_api_key = os.environ["OPENAI_API_KEY"]

# llm = ChatOpenAI(model="gpt-3.5-turbo")
llm = ChatOpenAI(model=MODEL_GPT)

parser = StrOutputParser()

system_template = "Translate the following into {language}:"

prompt_template = ChatPromptTemplate.from_messages([
    ('system', system_template),
    ('user', '{text}')
])

chain = prompt_template | llm | parser

app = FastAPI(
  title="simpleTranslator",
  version="1.0",
  description="A simple API server using LangChain's Runnable interfaces",
)

add_routes(
    app,
    chain,
    path="/chain",
)

# [RuntimeError: asyncio.run() cannot be called from a running event loop]
# if __name__ == "__main__":
#     import uvicorn

#     uvicorn.run(app, host="localhost", port=8000)

if __name__ == "__main__":
    import uvicorn
    
    config = uvicorn.Config(app, host="localhost", port=8000)
    server = uvicorn.Server(config)
    await server.serve()  # Use inside an async function

INFO:     Started server process [16248]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://localhost:8000 (Press CTRL+C to quit)



     __          ___      .__   __.   _______      _______. _______ .______     ____    ____  _______
    |  |        /   \     |  \ |  |  /  _____|    /       ||   ____||   _  \    \   \  /   / |   ____|
    |  |       /  ^  \    |   \|  | |  |  __     |   (----`|  |__   |  |_)  |    \   \/   /  |  |__
    |  |      /  /_\  \   |  . `  | |  | |_ |     \   \    |   __|  |      /      \      /   |   __|
    |  `----./  _____  \  |  |\   | |  |__| | .----)   |   |  |____ |  |\  \----.  \    /    |  |____
    |_______/__/     \__\ |__| \__|  \______| |_______/    |_______|| _| `._____|   \__/     |_______|
    
LANGSERVE: Playground for chain "/chain/" is live at:
LANGSERVE:  │
LANGSERVE:  └──> /chain/playground/
LANGSERVE:
LANGSERVE: See all available routes at /docs/
INFO:     ::1:1748 - "GET /chain/playground/ HTTP/1.1" 200 OK
INFO:     ::1:1748 - "GET /chain/playground/assets/index-400979f0.js HTTP/1.1" 200 OK
INFO:     ::1:1749 - "GET /chain/playground/assets/index-52e8ab2f.css HTTP

INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [16248]


### Run server
* In terminal:
    * python simpleTranslator.py
* Check app in browser:
    * [http://localhost:8000/](http://localhost:8000/)

### Try app using "toy demo UI" provided by LangServe Playground
* Try app in browser:
    * [http://localhost:8000/chain/playground/](http://localhost:8000/chain/playground/) 

### We can use LangServe Playground from notebook

In [63]:
from langserve import RemoteRunnable

remote_chain = RemoteRunnable("http://localhost:8000/chain/")
remote_chain.invoke({"language": "Spanish", "text": "Generative AI is a bigger opportunity than Internet"})

'La inteligencia artificial generativa es una oportunidad más grande que Internet.'